## K-Means Algorithm 

### Importance of the Similarity Measure

### Convergence Test

### Relationship of k-means with GMM

### Predictions

In [16]:
%matplotlib inline
import numpy as np 
import sklearn.preprocessing
import sklearn.datasets
import pandas as pd
import sklearn.model_selection
import numpy.random
import math
import sklearn.metrics
numpy.random.seed(42)

In [17]:
X, y = sklearn.datasets.load_iris(return_X_y=True)
#X, y = sklearn.datasets.load_wine(return_X_y=True)

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=42)
standard = sklearn.preprocessing.StandardScaler()
X_train = standard.fit_transform(X_train)
training_data = np.c_[X_train, y_train]#All of the features are continuous, so, no need to use one-hot encoder and we can directly standard normalize the features of the data set

X_test = standard.transform(X_test)
test_data = np.c_[X_test, y_test]
print(training_data.shape)
print(test_data.shape)
k = len(set(y_train))
y_train#It needs to be labeled from 0 to k


(112, 5)
(38, 5)


array([0, 0, 2, 1, 1, 0, 0, 1, 2, 2, 1, 2, 1, 2, 1, 0, 2, 1, 0, 0, 0, 1,
       2, 0, 0, 0, 1, 0, 1, 2, 0, 1, 2, 0, 2, 2, 1, 1, 2, 1, 0, 1, 2, 0,
       0, 1, 1, 0, 2, 0, 0, 1, 1, 2, 1, 2, 2, 1, 0, 0, 2, 2, 0, 0, 0, 1,
       2, 0, 2, 2, 0, 1, 1, 2, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1,
       2, 2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2, 1, 2, 1, 1, 2, 2, 0, 1, 2, 0,
       1, 2])

In [18]:
class kmeans(object):

    def __init__(self, X_train, k):
        self.K = k
        self.m = X_train.shape[0]
        self.n = X_train.shape[1]
        self.X_train = X_train
        choices = numpy.random.choice(np.arange(0, self.m), self.K, replace=False)
        self.centers = [X_train[choices[i], :].reshape(-1, 1) for i in range(0, self.K)]# initalize the clusters centers to be one of the observations
        self.clusters_assignments = np.zeros((self.m, 1))#Just to give it the necessary shape
    
    def reassign_clusters_centers(self):
        for k in range(0, self.K):
            temp = np.zeros((self.n, 1))
            clusters = list(map(lambda i: True if i == k else False, self.clusters_assignments))
            #for i in range(0, self.m):
                #if clusters[i] == True:
                    #temp += clusters[i] * self.X_train[i, :].reshape(-1, 1)#clusters contained in {0, 1}
                #else:
                    #pass
            clusters = np.array([clusters]*self.n)
            temp = np.sum( clusters.T *1 *self.X_train, axis=0)
            #print(np.sum(clusters))
            self.centers[k] = temp/(np.sum(clusters) + 1)

    def distortiuon_function(self):
        temp = 0
        for i in range(0, self.m):
            for k in range(0, self.K):
                if self.clusters_assignments[i] == k:
                    temp += np.linalg.norm(self.X_train[i, :].reshape(-1, 1) - self.centers[k].reshape(-1, 1))**2
                    break#They willn't be assigned to more than one cluster in tandem
        return temp

    def assign_to_clusters(self, x):
        temp = []
        for k in range(0, self.K):
            temp.append(np.linalg.norm(x.reshape(-1, 1) - self.centers[k].reshape(-1, 1))**2)#We will use L2-norm for dissimilarity measure
        return np.argmin(temp)#return the cluster number

    def E_step(self):
        for i in range(0, self.m):
            self.clusters_assignments[i] = self.assign_to_clusters(self.X_train[i, :].reshape(-1, 1))
    
    def fit(self, max_iterations, eps=1e-5):
        self.E_step()#To initialize the clusters assignments
        past = 10
        future = 0
        count = 0
        while(abs(past - future) > eps):#I will care for only lack of progress because k-means will always be able to minimize the distortion functions
            print(f"count:{count}, max_iterations{max_iterations}, past:{past}, future:{future}")
            count += 1
            past = self.distortiuon_function()
            self.reassign_clusters_centers()#The M step
            self.E_step()
            future = self.distortiuon_function()

        return self.centers, self.clusters_assignments

    def prediction_dataset(self, X):
        predictions = []
        for i in range(0, X.shape[0]):
            predictions.append(self.assign_to_clusters(X[i, :].reshape(-1, 1)))
        return predictions

    def predict(self, x):
        return  self.assign_to_clusters(x.reshape(-1, 1))



In [24]:
model = kmeans(X_train, k)
centers, clusters  = model.fit(1000)

count:0, max_iterations1000, past:10, future:0
count:1, max_iterations1000, past:197.34020393182593, future:317.2190907121861
count:2, max_iterations1000, past:317.2190907121861, future:312.67840672555525
count:3, max_iterations1000, past:312.67840672555525, future:303.49303093005165
count:4, max_iterations1000, past:303.49303093005165, future:301.41359418115564
count:5, max_iterations1000, past:301.41359418115564, future:301.68071435289903


In [25]:
pred = model.prediction_dataset(X_train)
#centers

In [26]:
pred = model.prediction_dataset(X_train)
print("Performance on the training set")
#print(sklearn.metrics.confusion_matrix(y_train, pred))
c = sklearn.metrics.confusion_matrix(y_train, pred)
c[:, list(np.argmax(c, axis=1))]#ordering the cluster to where it shows the highest number of matching with the true labels

Performance on the training set


array([[35,  0,  0],
       [ 0, 22, 17],
       [ 0,  3, 35]], dtype=int64)

In [27]:
pred = model.prediction_dataset(X_test)
print("Performance on the test set")
#print(sklearn.metrics.confusion_matrix(y_test, pred))
c = sklearn.metrics.confusion_matrix(y_test, pred)
c[:, list(np.argmax(c, axis=1))]#ordering the cluster to where it shows the highest number of matching with the true labels

Performance on the test set


array([[15,  0,  0],
       [ 0,  6,  6],
       [ 0, 12, 12]], dtype=int64)

In [29]:
import sklearn.cluster
model = sklearn.cluster.KMeans(n_clusters=3, max_iter=2000)
model.fit(X_train)
pred = model.predict(X_train)
print("Performance on the train set")
#print(sklearn.metrics.confusion_matrix(y_test, pred))
c = sklearn.metrics.confusion_matrix(y_train, pred)
c[:, list(np.argmax(c, axis=1))]#ordering the cluster to where it shows the highest number of matching with the true labels

Performance on the train set


array([[35,  0,  0],
       [ 0, 32,  7],
       [ 0, 14, 24]], dtype=int64)

In [30]:
a = np.array([[1, 2, 3,4], [4, 5, 6, 7]])
np.sum(a, axis=0)

array([ 5,  7,  9, 11])

### References 
* Chapter 1, chapter 2 and Chapter 9 from Bishop, C. (2006). Pattern Recognition and Machine Learning. Cambridge: Springer.
* Andrew Ng, Lec 12: (https://www.youtube.com/watch?v=ZZGTuAkF-Hw)

